# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

Version 24.1.0 of Howso Engine™ is available. You are using version 23.1.1.dev2+gaa28aac.d20240625.
The following parameters from configuration file will override the Amalgam parameters set in the code: {'library_path', 'trace'}


,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions_robust": True,
        "feature_residuals_robust": True,
        "local_case_feature_residual_convictions_robust": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.13707084604413974,
            'Iris-virginica': 0.8629291539558602}},
 {'class': {'Iris-virginica': 1}}, {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 0.3747821071550217,
            'Iris-virginica': 0.6252178928449784}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 0.8279595415790274,
            'Iris-virginica': 0.17204045842097254}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-virginica': 1}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.20312938217244803,
            'Iris-virginica': 0.796870617827552}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.19088492679898608,
            'Iris-virginica': 0.809115073201014}},
 {'class': {'Iris-virginica': 1}},
 {'cl

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-versicolor': 0.3747821071550217,
  'Iris-virginica': 0.6252178928449784}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [7]:
inf_cases_4 = pd.DataFrame(reaction["details"]["influential_cases"][4])
inf_cases_4

,petal length,sepal length,class,petal width,.session,.session_training_index,sepal width,.influence_weight
0,5.1,6.3,Iris-virginica,1.5,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,99,2.8,0.254831
1,4.9,6.3,Iris-versicolor,1.5,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,107,2.5,0.212308
2,5.0,6.0,Iris-virginica,1.5,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,34,2.2,0.189139
3,5.6,6.3,Iris-virginica,1.8,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,61,2.9,0.181248
4,4.7,6.1,Iris-versicolor,1.4,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,89,2.9,0.162475


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [8]:
inf_cases_20 = pd.DataFrame(reaction["details"]["influential_cases"][20])
inf_cases_20

,petal length,sepal length,class,petal width,.session,.session_training_index,sepal width,.influence_weight
0,1.4,5.1,Iris-setosa,0.3,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,48,3.5,0.214285
1,1.6,5.0,Iris-setosa,0.4,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,63,3.4,0.209768
2,1.7,5.1,Iris-setosa,0.5,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,116,3.3,0.195387
3,1.7,5.4,Iris-setosa,0.2,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,9,3.4,0.191825
4,1.4,5.2,Iris-setosa,0.2,a4c2f366-8cb3-4e6c-8885-3e90f3d833ee,43,3.4,0.188735


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [9]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
2,6.0,2.2,5.0,1.5,Iris-virginica,0.579978
1,6.3,2.5,4.9,1.5,Iris-versicolor,0.794229
0,6.3,2.8,5.1,1.5,Iris-virginica,0.847696
3,6.3,2.9,5.6,1.8,Iris-virginica,1.082284
4,6.1,2.9,4.7,1.4,Iris-versicolor,1.096176


In [10]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
2,5.1,3.3,1.7,0.5,Iris-setosa,0.794620
3,5.4,3.4,1.7,0.2,Iris-setosa,0.890655
1,5.0,3.4,1.6,0.4,Iris-setosa,1.007523
0,5.1,3.5,1.4,0.3,Iris-setosa,1.132817
4,5.2,3.4,1.4,0.2,Iris-setosa,1.324839


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [11]:
fcs_13 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][13:14], index=[13])
fcs_20 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][20:21], index=[20])

display(fcs_13)
display(fcs_20)

,petal length,sepal length,petal width,sepal width
13,0.259045,0.16665,0.41521,0.087717


,petal length,sepal length,petal width,sepal width
20,0.161386,0.127858,0.16544,0.099724


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [12]:
pd.DataFrame(reaction["details"]["feature_residuals_robust"])

,petal length,sepal length,class,petal width,sepal width
0,0.374060,0.417472,0.232853,0.193922,0.202079
1,0.405746,0.348620,0.306859,0.249355,0.254013
2,0.441464,0.327248,0.136578,0.293589,0.152011
3,0.448006,0.383943,0.315642,0.297862,0.266241
4,0.397378,0.366351,0.308851,0.240998,0.216862
5,0.320513,0.327379,0.112297,0.135037,0.163985
6,0.416669,0.372882,0.091492,0.202498,0.252078
7,0.432408,0.354586,0.238822,0.279099,0.173340
8,0.355315,0.341950,0.208509,0.159849,0.236482
9,0.364841,0.331536,0.124859,0.142175,0.243108


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [13]:
pd.DataFrame(reaction["details"]["local_case_feature_residual_convictions_robust"])

,petal length,sepal length,class,petal width,sepal width
0,0.884603,0.764322,1.102658,1.105519,1.844925
1,0.833007,1.231538,1.325791,1.227720,0.603092
2,0.952672,1.057963,0.937551,0.693767,2.419911
3,1.049806,0.577990,0.648884,0.961220,0.607771
4,0.589318,0.845580,0.527353,0.514133,0.637624
5,1.727538,1.314625,0.888319,1.953066,0.861602
6,1.009293,1.554665,0.833265,1.134155,0.649338
7,0.731796,1.036696,0.802297,1.899220,0.778646
8,1.123844,0.589618,1.148352,1.016240,0.393193
9,1.056185,0.973745,0.749322,0.932744,0.538104


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [14]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[1.7427848961221073, 1.2266775314557734, 1.2832957000238383, 1.1358200640851237,
 0.7663033973777346, 1.7093000149088442, 1.62988622348009, 1.179424858114735,
 0.7771310371991803, 1.1877899178467337, 2.729516377134626, 1.2638120732438078,
 4.6379323779451465, 1.2530048620888574, 1.0950665861958442, 1.3407402153095342,
 1.0767771072504961, 1.9509548748596433, 0.8490545593829374, 0.5167812597435596,
 0.9554756570604226, 1.6330745670443925, 0.7713719862798815, 1.4839431656699253,
 1.6248239296155955, 1.7357489473788903, 1.2057177277105207, 1.3955068084226425,
 1.036532322552324, 1.197977154227804]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.